# Import dependecies 

In [6]:
import numpy as np
import pandas as pd
 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
#from xgboost import XGBClassifier
from sklearn import metrics

# Get stock data

In [14]:
url="https://raw.githubusercontent.com/plotly/datasets/master/tesla-stock-price.csv"
df=pd.read_csv(url,on_bad_lines='skip')
get_stock_data = df.iloc[1:]
get_stock_data.copy()

,date,close,volume,open,high,low
1,2018/10/15,259.59,6189026.0000,259.06,263.2800,254.5367
2,2018/10/12,258.78,7189257.0000,261.00,261.9900,252.0100
3,2018/10/11,252.23,8128184.0000,257.53,262.2500,249.0300
4,2018/10/10,256.88,12781560.0000,264.61,265.5100,247.7700
5,2018/10/09,262.80,12037780.0000,255.25,266.7700,253.3000
...,...,...,...,...,...,...
752,2015/10/21,210.09,4177956.0000,211.99,214.8100,208.8000
753,2015/10/20,213.03,14877020.0000,227.72,228.6000,202.0000
754,2015/10/19,228.10,2506836.0000,226.50,231.1500,224.9400
755,2015/10/16,227.01,4327574.0000,223.04,230.4805,222.8700


# Clean Data

In [17]:
splitted = get_stock_data['date'].str.split('/', expand=True)
 
get_stock_data['day'] = splitted[1].astype('int')
get_stock_data['month'] = splitted[0].astype('int')
get_stock_data['year'] = splitted[2].astype('int')

get_stock_data['is_quarter_end'] = np.where(get_stock_data['month']%3==0,1,0)

 
data_cleaning = get_stock_data.copy()

/var/folders/zk/m4ts23_n5376402hsx6_0fk00000gn/T/ipykernel_21457/2571188718.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  get_stock_data['day'] = splitted[1].astype('int')
/var/folders/zk/m4ts23_n5376402hsx6_0fk00000gn/T/ipykernel_21457/2571188718.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  get_stock_data['month'] = splitted[0].astype('int')
/var/folders/zk/m4ts23_n5376402hsx6_0fk00000gn/T/ipykernel_21457/2571188718.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

# Feature Engineering

In [19]:
data_cleaning['open-close'] = data_cleaning['open'] - data_cleaning['close']
data_cleaning['low-high'] = data_cleaning['low'] - data_cleaning['high']
data_cleaning['target'] = np.where(data_cleaning['close'].shift(-1) > data_cleaning['close'], 1, 0)

feature_engineering = data_cleaning.copy()

# Scale Split

In [29]:
# name cell scale_split


features = feature_engineering[['open-close', 'low-high', 'is_quarter_end']]
target = feature_engineering['target']
 
scaler = StandardScaler()
features = scaler.fit_transform(features)
 
X_train, X_valid, Y_train, Y_valid = train_test_split(
    features, target, test_size=0.1, random_state=2022)

#return X_train, X_valid, Y_train, Y_valid

# Train/Eval

In [27]:
# uncomment line:
#X_train, X_valid, Y_train, Y_valid = scale_split

models = [LogisticRegression(), SVC(
  kernel='poly', probability=True)]
 
for i in range(2):
  models[i].fit(X_train, Y_train)
 
  print(f'{models[i]} : ')
  print('Training Accuracy : ', metrics.roc_auc_score(
    Y_train, models[i].predict_proba(X_train)[:,1]))
  print('Validation Accuracy : ', metrics.roc_auc_score(
    Y_valid, models[i].predict_proba(X_valid)[:,1]))
  print()

LogisticRegression() : 
Training Accuracy :  0.9117993079584775
Validation Accuracy :  0.9256944444444445

SVC(kernel='poly', probability=True) : 
Training Accuracy :  0.9093555363321799
Validation Accuracy :  0.9138888888888889

